In [ ]:
# Importing some General Libraries 
import numpy as np
import pyshtools as shtools

In [ ]:
# Tweaking $PYTHONPATH so we find TOMOPROXY
import sys
import os, glob
sys.path.append('../')

In [ ]:
# Importing tomoproxy modules
import tomoproxy.plotting
import tomoproxy.sh_tools
import tomoproxy.geody_compare as gc

In [ ]:
# List of Compositions, pPv Scenarios and Mineralogical Models
comp_list = ['pyrolite', 'BMO', 'MORB', 'HC']
ppv_list = ['noppv', 'ppv', 'partppv']
min_model_list = ['SLB_2022', 'SLB_2011']

In [ ]:
# Read input file
input_path = '../data/D12_models/'

# depth
depth = np.loadtxt(input_path + 'D12_4000_depth_layers.dat')[:,1]
depth = depth[14:] # Truncate upper mantle layers

# coordinates
coords = np.loadtxt(input_path + 'D12_4000_lonlat.dat')
lon = coords[:,0] % 360
lat = coords[:,1]

In [ ]:
# Input and create Layers
i = 1

t_grid = {'TC': np.zeros((len(depth),len(lat))),
          'TH': np.zeros((len(depth),len(lat)))}
comp_grid = np.zeros_like(t_grid['TC'])

for filename in glob.glob(os.path.join(input_path, 'D12-TC_4000/*.dat')):
    loc = int(os.path.splitext(filename)[0].split('_')[-1]) - 1
    if loc >= 14:
        data1 = np.loadtxt(filename)
        loc -= 14
    else:
        continue

    t_grid['TC'][loc] = data1[:,0]
    comp_grid[loc] = data1[:,2]
    i += 1

for filename in glob.glob(os.path.join(input_path, 'D12-TH_4000/*.dat')):
    loc = int(os.path.splitext(filename)[0].split('_')[-1]) - 1
    if loc >= 14:
        data1 = np.loadtxt(filename)
        loc -= 14
    else:
        continue

    t_grid['TH'][loc] = data1[:,0]
    i += 1

In [ ]:
# Create properties for compositions and ppv
def find_comp_properties(comp):
    comp_all = ['pyrolite', 'pyroliteTC', 'BMO', 'MORB', 'HC']
    comp_index = comp_all.index(comp)
    t_grid = ['TH', 'TC', 'TC', 'TC', 'TC']
    X = [None, None, comp_grid, comp_grid, comp_grid]

    return t_grid[comp_index], X[comp_index]

def find_ppv_type(ppv):
    ppv_type = ['none', 'two_phase', None]
    ppv_all = ['noppv', 'ppv', 'partppv']
    ppv_index = ppv_all.index(ppv)

    return ppv_type[ppv_index]

In [ ]:
# Generate TwoPhaseRegion Objects
save = True
load = True

phaseboundary = {}
for min_model in min_model_list:
    for comp in comp_list:
        name = f'{comp}_{min_model[-2:]}'
        if load:
            phaseboundary[name] = gc.BdgPPvTwoPhaseRegion.from_txt(os.path.join(input_path, f'ppv_two_phase_boundary_{name}'))
        else:
            phaseboundary[name] = gc.BdgPPvTwoPhaseRegion(comp, min_model=min_model, save=save, outdir=input_path)

In [ ]:
# Convert Oxide to Phase
save = True
load = True

phases = {}
for min_model in min_model_list:
    for comp in ['pyroliteTC'] + comp_list:  
        name = f'{comp}_{min_model[-2:]}'
        t, X_type = find_comp_properties(comp)
        if load:
            filename = os.path.join(input_path, f'phases_{comp}_{min_model[-2:]}.npz')
            phases[name] = gc.PhaseGrid(filename, t_grid[t], depth, lon, lat, comp, min_model)

        else:
            phases[name] = gc.oxide_to_phase(t_grid[t], depth, lon, lat, comp, phaseboundary[name], X=X_type, min_model=min_model, save=save, outdir=input_path)

In [ ]:
# Calculate Elastic Parameters and create ElasticGrids
save = True    

elastic = {}
for min_model in min_model_list:
    for comp in ['pyroliteTC'] + comp_list:  
        phase_name = f'{comp}_{min_model[-2:]}'
        _, X_type = find_comp_properties(comp)

        for ppv in ppv_list:
            ppv_type = find_ppv_type(ppv)
            elastic_name = f'{comp}_{ppv}_{min_model[-2:]}'

            if ppv == 'partppv':
                elastic[elastic_name] = gc.ElasticGrid.from_file(input_path, comp, 'partppv', depth, lon, lat, min_model=min_model, comp_grid=comp_grid)
            elif load:
                elastic[elastic_name] = gc.ElasticGrid.from_file(input_path, comp, ppv, depth, lon, lat, min_model=min_model)
            else:
                if X_type is not None:
                    py_model = elastic[f'pyroliteTC_{ppv}_{min_model[-2:]}']
                else:
                    py_model = None
                elastic[elastic_name] = phases[phase_name].evaluate_elastic(ppv_type, X=X_type, py_model=py_model, save=save, outdir=input_path)



In [ ]:
# Load ElasticGrids
for min_model in min_model_list:
    for comp in comp_list:
        for ppv in ppv_list:
            if ppv == 'partppv':
                X = comp_grid
            else:
                X = None
            name = f'{comp}_{ppv}_{min_model[-2:]}'
            elastic{name} = gc.ElasticGrid.from_file(input_path, comp, ppv, depth, lon, lat, min_model=min_model, comp_grid=X)

In [ ]:
# Calculate spherical harmonics and create/load RawSeismicModels
save = True
load = True
spherical_degree = 8
radial_degree = 20 

raw_velocity = {}
for min_model in min_model_list:
    for comp in comp_list:
        for ppv in ppv_list:
            name = f'{comp}_{ppv}_{min_model[-2:]}'
            if load:
                raw_velocity{name} = gc.RawSeismicModel.from_file(radial_degree, input_path, 
                                                                  comp, ppv, min_model, seismic_model='SOLA_')
            else:
                raw_velocity{name} = elastic{name}.to_continuous_param(r_deg=radial_degree, sph_deg=spherical_degree, 
                                                                       save=save, outdir=input_path, 
                                                                       filename=f'SOLA_{name}')

In [ ]:
# Generating Filtered Seismic Models

models = {}
for min_model in min_model_list:
    for comp in comp_list:
        for ppv in ppv_list:
            name = f'{comp}_{ppv}_{min_model[-2:]}'
            models{name} = raw_velocity{name}.to_SOLA()

            models{name}.apply_kernel()

# Calculate distributions of tomographic characteristics
Here, we obtain distributions of RMS $d\ln V_p$, RMS $d\ln V_s$, $R_{s/p}$, $r_{s–c}$ to get uncertainties of these tomographic characteristics 

In [ ]:
# Functions to calculate R_s/p and r_s-c
def rms(array, ax = None, data_type = None):
    """
    Calculates the root mean square (RMS) of values in an array.
    If the array is a set of spherical coefficients, set 
    data_type = 'SH'.
    """
    if data_type == "SH":
        return np.sqrt(np.sum((array ** 2) / (4 * np.pi), axis = ax))
    else:
        return np.sqrt(np.sum((array ** 2) / np.size(array), axis = ax))
    
    
def SOLA_correlate(v1, v2, deg, omit_zero = False):
    """
    Correlates two sets of spherical harmonics v1 and v2.
    """

    corr = []

    if omit_zero:
        f = 1
    else:
        f = 0

    # loop through each layer, correlation assumes lower resolution of the two
    cross = shtools.spectralanalysis.cross_spectrum(v1, v2,
                            degrees=deg,
                            normalization='ortho')
    a = shtools.spectralanalysis.spectrum(v1, 
                            degrees=deg,
                            normalization='ortho')
    b = shtools.spectralanalysis.spectrum(v2,
                            degrees=deg,
                            normalization='ortho')
    corr = np.sum(cross[f:])/np.sqrt(np.sum(a[f:])*np.sum(b[f:]))    
    
    return corr 


def calculate_R(dvs, dvp, lat, lon, threshold = 0.1, deg = None):
    """
    Calculate the average $R_{s/p} = d \ln V_s / d\ln V_p$ over a depth slice by two methods: 
    root mean square in grid space or by spherical harmonics.
    """
    SH = rms(dvs, data_type='SH')/rms(dvp, data_type='SH')

    grid_vs = make_grids_with_deg_filter(dvs, lat, lon, deg = deg)
    grid_vp = make_grids_with_deg_filter(dvp, lat, lon, deg = deg)

    root = rms(grid_vs[(abs(grid_vs) > threshold) & (abs(grid_vp) > threshold)])/rms(grid_vp[(abs(grid_vs) > threshold) & (abs(grid_vp) > threshold)])

    return root, SH


def make_grids_with_deg_filter(coefs, lat, lon, deg = None):
    """
    Transform spherical coefficients (coefs) to real space, as determined by a list of latitudes (lat)
    and longitudes (lon).
    """
    new_coefs = np.zeros_like(coefs)

    if deg != None:
        for d in deg:
            new_coefs[:,d] = coefs[:,d]
    else:
        new_coefs = coefs

    return shtools.expand.MakeGridPoint(new_coefs, lat, lon, norm = 4)


In [ ]:
# Importing SOLA Tomographic Model
SOLA_path = "../data/SOLA_model/"
SOLA_model = gc.SOLAShell.from_directory(SOLA_path)
SOLA_model.apply_resolving_kernel()

SOLA_model.vp = tomoproxy.sh_tools.rts_to_sh(SOLA_model.vp) / 100
SOLA_model.vs = tomoproxy.sh_tools.rts_to_sh(SOLA_model.vs) / 100
SOLA_model.vphi = tomoproxy.sh_tools.rts_to_sh(SOLA_model.vphi) / 100
SOLA_model.vp_err = tomoproxy.sh_tools.rts_to_sh(SOLA_model.vp_err) / 100
SOLA_model.vs_err = tomoproxy.sh_tools.rts_to_sh(SOLA_model.vs_err) / 100
SOLA_model.vphi_err = tomoproxy.sh_tools.rts_to_sh(SOLA_model.vphi_err) / 100

In [ ]:
# Generate and save Uncertainties on R and Vs-Vc with SH
save = True
load = True

if load:
    a = np.load(input_path + 'SOLA_model_realisations.npz',)
    SOLA_Vp_layer_rand = a['vp_random']
    SOLA_Vs_layer_rand = a['vs_random']
    SOLA_Vphi_layer_rand = a['vphi_random']
    R_random = a['R_random']
    corr_random = a['corr_random']

    Vp_random = rms(SOLA_Vp_layer_rand, ax = (1,2,3), data_type = 'SH')
    Vs_random = rms(SOLA_Vs_layer_rand, ax = (1,2,3), data_type = 'SH')
    Vphi_random = rms(SOLA_Vphi_layer_rand, ax = (1,2,3), data_type = 'SH')
else:
    np.random.seed(12345)
    SOLA_Vp_layer_rand = np.random.normal(SOLA_model.vp, np.abs(SOLA_model.vp_err), (1000000,2,9,9))

    np.random.seed(93720)
    SOLA_Vs_layer_rand = np.random.normal(SOLA_model.vs, np.abs(SOLA_model.vs_err), (1000000,2,9,9))

    np.random.seed(36183)
    SOLA_Vphi_layer_rand = np.random.normal(SOLA_model.vphi, np.abs(SOLA_model.vphi_err), (1000000,2,9,9))

    Vp_random = rms(SOLA_Vp_layer_rand, ax = (1,2,3), data_type= = 'SH')
    Vs_random = rms(SOLA_Vs_layer_rand, ax = (1,2,3), data_type = 'SH')
    Vphi_random = rms(SOLA_Vphi_layer_rand, ax = (1,2,3), data_type = 'SH')
    R_random = Vs_random/Vp_random
    corr_random = [SOLA_correlate(SOLA_Vphi_layer_rand[i],SOLA_Vs_layer_rand[i], deg = np.arange(0,9,1), omit_zero = True) for i in range(1000000)]

    if save:
        np.savez(os.path.join(input_path, 'SOLA_model_realisations'),
                vp_random = SOLA_Vp_layer_rand,
                vs_random = SOLA_Vs_layer_rand,
                vphi_random = SOLA_Vphi_layer_rand,
                R_random = R_random,
                corr_random = corr_random
                )

In [ ]:
# Import equidistant grid
eq_dist = np.loadtxt('../deps/see-rts-filtering/data_files/SP12RTS.1x1/slices.eqdst.1/SP12RTS..EC.0025.eqdst.1.latlon.dat', usecols = (0,1))

In [ ]:
# Code to calculate R (only run if files are not downloaded)
root = np.zeros((24, 9))
SH = np.zeros_like(root)

save = True
load = True

if load:
    a = np.load(input_path + 'R_values.npz')
    root = a['root']
    SH = a['SH']
else:
    for i, m in enumerate(models.values()):
        root[i, 0], SH[i, 0] = calculate_R(m.vs, m.vp, eq_dist[:,0], eq_dist[:,1])
        for j in range(1,9):
            root[i, j], SH[i, j] = calculate_R(m.vs, m.vp, eq_dist[:,0], eq_dist[:,1], deg = [j])

    if save:
        np.savez(input_path + 'R_values',
                root = root,
                SH = SH
                )

In [ ]:
# Code to calculate S-C correlation (only run if files are not downloaded)
save = True
load = True

if load:
    a = np.load(input_path + 'vphi_vs_corr.npz')
    vphi_vs_corr = a['vphi_vs_corr']
else:
    vphi_vs_corr = np.zeros((24, 9))
    for i, m in enumerate(models):
        vphi_vs_corr[i,0] = SOLA_correlate(m.vphi, m.vs, np.arange(0,9,1), omit_zero = True)
        for j in range(1,9):
            vphi_vs_corr[i,j] = SOLA_correlate(m.vphi, m.vs, [j], omit_zero = False)
    if save:
        np.savez(input_path + 'vphi_vs_corr',
                vphi_vs_corr = vphi_vs_corr)

In [ ]:
# For original and reparameterised models
load = True
save = True

if load:
    a = np.load(input_path + 'R_and_corr_terra_and_reparam.npz',)
    root_terra = a['root_terra']
    vphi_vs_corr_terra = a['vphi_vs_corr_terra']
else:
    depths = 6371 - depth[-20:]
    threshold = 0.1
    root_terra = np.zeros((24, len(depths)))
    root_reparam = np.zeros((24,len(depths), 9))
    SH_reparam = np.zeros_like(root_reparam)

    vphi_vs_corr_terra = np.zeros((24, len(depths)))
    vphi_vs_corr_reparam = np.zeros((24, len(depths), 9))

    comp_list = ['pyrolite', 'BMO', 'MORB', 'HC']
    ppv_list = ['noppv', 'ppv', 'partppv']
    min_model_list = ['SLB_2022', 'SLB_2011']


    for min_model in min_model_list:
        for comp in comp_list:  
            for ppv in ppv_list:
                name = f'{comp}_{ppv}_{min_model[-2:]}'
                
        terra_model = elastic[name]
        sshell = raw_velocity[name]

        for k, d in enumerate(depths):
            index = -20 + k
            vs_shell = sshell.vs.get_sh_coefs_at_r(d)
            vp_shell = sshell.vp.get_sh_coefs_at_r(d)
            vphi_shell = sshell.vphi.get_sh_coefs_at_r(d)
            t_s = 100*(terra_model.vs_grid[index] - vs_shell[0,0,0] / (2.0 * np.sqrt(np.pi))) / (vs_shell[0,0,0] / (2.0 * np.sqrt(np.pi)))
            t_p = 100*(terra_model.vp_grid[index] - vp_shell[0,0,0] / (2.0 * np.sqrt(np.pi))) / (vp_shell[0,0,0] / (2.0 * np.sqrt(np.pi)))
            t_phi = 100*(terra_model.vphi_grid[index] - vphi_shell[0,0,0] / (2.0 * np.sqrt(np.pi))) / (vp_shell[0,0,0] / (2.0 * np.sqrt(np.pi)))
            vs_shell /= vs_shell[0,0,0] / (2.0 * np.sqrt(np.pi))
            vs_shell[0,0,0] = 0
            vp_shell /= vp_shell[0,0,0] / (2.0 * np.sqrt(np.pi))
            vp_shell[0,0,0] = 0
            vphi_shell /= vphi_shell[0,0,0] / (2.0 * np.sqrt(np.pi))
            vphi_shell[0,0,0] = 0

            root_terra[i,k] = rms(t_s[(abs(t_s) > threshold) & (abs(t_p) > threshold)]) / rms(t_p[(abs(t_s) > threshold) & (abs(t_p) > threshold)])
            vphi_vs_corr_terra[i,k] = np.corrcoef(t_s, t_phi)[0,1]

            root_reparam[i, k, 0], SH_reparam[i,k, 0] = calculate_R(100*vs_shell, 100*vp_shell, eq_dist[:,0], eq_dist[:,1])
            vphi_vs_corr_reparam[i,k,0] = SOLA_correlate(100*vphi_shell, 100*vs_shell, np.arange(0,9,1), omit_zero = True)

            for j in range(1,9):
                root_reparam[i, k, j], SH_reparam[i, k, j] = calculate_R(100*vs_shell, 100*vp_shell, eq_dist[:,0], eq_dist[:,1], deg = [j])
                vphi_vs_corr_reparam[i,k,j] = SOLA_correlate(100*vphi_shell, 100*vs_shell, [j], omit_zero = False)
        
    if save:
        np.savez(input_path + 'R_and_corr_terra_and_reparam',
                root_terra = root_terra,
                root_reparam = root_reparam,
                SH_reparam = SH_reparam,
                vphi_vs_corr_terra = vphi_vs_corr_terra,
                vphi_vs_corr_reparam = vphi_vs_corr_reparam)


/var/folders/cs/4dd1zds94b5c7zcgwb3dbv3c0000gq/T/ipykernel_78040/3716363753.py:20: RuntimeWarning: invalid value encountered in scalar divide
  root = rms(grid_vs[(abs(grid_vs) > threshold) & (abs(grid_vp) > threshold)])/rms(grid_vp[(abs(grid_vs) > threshold) & (abs(grid_vp) > threshold)])
/var/folders/cs/4dd1zds94b5c7zcgwb3dbv3c0000gq/T/ipykernel_78040/3867904830.py:39: RuntimeWarning: invalid value encountered in scalar divide
  root_terra[i,k] = rms(t_s[(abs(t_s) > threshold) & (abs(t_p) > threshold)]) / rms(t_p[(abs(t_s) > threshold) & (abs(t_p) > threshold)])
/Users/univ4732/anaconda3/envs/lema/lib/python3.9/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/Users/univ4732/anaconda3/envs/lema/lib/python3.9/site-packages/numpy/lib/function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
